In [ ]:
%matplotlib ipympl
import numpy as np
from numpy.fft import  ifft2, ifftshift
from numpy.random import default_rng as randn
#from scipy.special import kv as beselk
from matplotlib import pyplot as plt
from ipywidgets import interact, widgets
from ows import ows
import time

rad2asec = 3600 * 180/np.pi
asec2rad = 1/rad2asec

## PSD
Power spectrum density

In [ ]:
# Input
dimmat = 128
dxp = 0.03222887597610881 # Pupil plane pixel size
r0 = .1
L0 = -1
D = 0.01
wl = 500e-9 # [m]

# Computing limit values
FoV = 25*asec2rad # arcsec
#Npx = 1000 # image plane pixels qty
#Daf = asec2rad*FoV#/Npx # [rad/px]  (Numerical aperture)
Daf = wl/(2*D) # max
dfp = Daf/wl # [/m] Pupil plane spatial frequency pixel size (Numerical aperture/WL)
dimpsf = 8*wl*np.sqrt((D/r0)**2 + 1)/(Daf*D)   # (min psf size) Matrix size
print("Miminum matrix dimention :",dimpsf)
# dimmat = int(dimpsf)+int((dimpsf %1 !=0)*1)
# dimmat += int((dimmat %2 != 0)*1)

if dimmat < dimpsf:
    print(f"dimmat is too small: {dimmat:d}. It souhld be greater than {dimpsf:.1f}")


PSD = ows.psd(dimmat, dxp, r0, wl)
#PSD[dimmat//2-1:dimmat//2+1,dimmat//2-1:dimmat//2+1] = 0
plt.close(1)
plt.figure(1)
plt.imshow(PSD**(1/8))
plt.title("Phase spectrum density ^(1/8)")
plt.colorbar()

## Phase screen & PSF

In [ ]:
SEED = 1

start = time.time()

[phase_screen, psf, pupil_mask, R] = ows.phase_screen(PSD, dxp, SEED, PUPIL = True, PD = [127,40])

end = time.time()


print("Processingtime :",
      (end-start) * 10**3, "ms")

plt.close(4)
plt.figure(4)
plt.imshow(phase_screen.real)
plt.title("phase screen")
plt.colorbar()

plt.show()
plt.close(5)
plt.figure(5)
plt.imshow((np.abs(psf)**(1/2)))
plt.title("PSF^(1/2)")
plt.colorbar()
plt.show()


# Lenslet illumination mask

In [ ]:
from astropy.io import fits

# def normalize(data):
#     min_val = np.min(data)
#     max_val = np.max(data)
#     return (data - min_val) / (max_val - min_val)

# phase_screen_normilized = normalize(phase_screen)
# psf_normalized = normalize(psf)

# fits.writeto('data/phase_screen.fits', phase_screen, overwrite=True)
# fits.writeto('data/phase_screen_normalized.fits', phase_screen_normilized, overwrite=True)
# fits.writeto('data/psf.fits', psf, overwrite=True)
# fits.writeto('data/psf_normalized.fits', psf_normalized, overwrite=True)

# input_infos = f"ro,{r0:f},m\nWL,{wl:g},m\ndxp,{dxp:f},\nseed,{1},"
# with open('data/inputs.txt', 'w') as f:
#     f.write(input_infos)

# Lenslet array illumination

In [ ]:
import itertools

N = 16
m = int(np.ceil(dimmat/N)) # By definition, the number of lenslets must be an integer
if N%2 != 0:
    padded_dim = m*(N) # N+1 in order to have a lenslet centered on the optical axis
else:
    padded_dim = m*(N+1)
print(f"Number of pixels per lenslets axis: {m}")

# Pad the pupil mask for illuminaiton computation
padded_mask = np.zeros((padded_dim,padded_dim),int)
padded_mask[(padded_dim-dimmat)//2:(padded_dim-dimmat)//2 + dimmat,(padded_dim-dimmat)//2:(padded_dim-dimmat)//2 + dimmat] = pupil_mask

padded_phase_screen = np.zeros_like(padded_mask,np.float64)

padded_phase_screen[(padded_dim-dimmat)//2:(padded_dim-dimmat)//2 + dimmat,(padded_dim-dimmat)//2:(padded_dim-dimmat)//2 + dimmat] = phase_screen


lenslet_mask = np.zeros_like(padded_mask)
lenslet_matrix = np.zeros((N,N))

ii=0
for i in range(0, padded_dim, m):
    jj=0
    for j in range(0, padded_dim, m):
        lenslet = padded_mask[i:i+m, j:j+m]
        if (lenslet.sum() >= (m*m)/2):
            lenslet_mask[i:i+m, j:j+m] = 1
            lenslet_matrix[ii,jj] = 1
        jj += 1
    ii +=1


plt.close(6)
plt.figure(6)
plt.subplot(2,2,1)
for i in range(1,N+1):
    x = [0, padded_dim] 
    y = [m*i, m*i] 
    plt.plot(x, y, 'darkred', linewidth=2)
for i in range(1,N+1):
    x = [m*i, m*i] 
    y = [0, padded_dim] 
    plt.plot(x, y, 'darkred', linewidth=2)
plt.imshow(padded_mask)
plt.title("Padded pupil mask")

plt.subplot(2,2,2)
for i in range(1,N+1):
    x = [0, padded_dim] 
    y = [m*i, m*i] 
    plt.plot(x, y, 'darkred', linewidth=2)
for i in range(1,N+1):
    x = [m*i, m*i] 
    y = [0, padded_dim] 
    plt.plot(x, y, 'darkred', linewidth=2)
plt.imshow(lenslet_mask)
plt.title("Padded Lenslet mask")

plt.subplot(2,2,3)
plt.imshow(lenslet_matrix)
plt.minorticks_on()
plt.grid(which='minor', linestyle='-', alpha=0.5)
plt.grid(which='major', linestyle='-', alpha=0.5)
plt.title("Padded lenslet matrix")
plt.show()

plt.close(7)
plt.figure(7)
for i in range(1,N+1):
    x = [0, padded_dim] 
    y = [m*i, m*i] 
    plt.plot(x, y, 'darkred', linewidth=2)
for i in range(1,N+1):
    x = [m*i, m*i] 
    y = [0, padded_dim] 
    plt.plot(x, y, 'darkred', linewidth=2)
plt.imshow(padded_phase_screen*lenslet_mask,cmap="grey")
plt.title("Padded phase screen\n masked for illumination >= 50% lenslet")
plt.show()

# Wavefront slope